<a href="https://colab.research.google.com/github/manrajc13/Byte_chase/blob/main/MCQ%2BCheatSheet%2BoutputPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PDF Parsing**
integrates OCR (Optical Character Recognition) with PDF text extraction to process a document and extract content, including text from images within the PDF.


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [ ]:
pip install pymupdf paddleocr langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
  Cre

In [ ]:
import os
import cv2
import numpy as np
import fitz  # PyMuPDF for rendering PDF pages as images
from paddleocr import PaddleOCR
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

In [ ]:
# Initializing PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Function to preprocess images incase of any present in the pdf
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) # it reades image from the given path in grayscale
    processed_image = cv2.adaptiveThreshold(
        image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    return processed_image

# Function to extract text from an image using PaddleOCR
def ocr_image(image_path):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    processed_image_path = image_path.replace(".png", "_processed.png")
    cv2.imwrite(processed_image_path, processed_image)

    # Perform OCR
    ocr_results = ocr.ocr(processed_image_path, cls=True)
     # Check if ocr_results is empty or None
    if ocr_results and ocr_results[0]:
        extracted_text = " ".join([line[1][0] for line in ocr_results[0]])  # Combine text lines
    else:
        extracted_text = ""  # Return an empty string if no text is detected

    return extracted_text # return the extracted text (if any) from the image found

# Main function for file processing
def file_processing(file, output_dir):
    os.makedirs(output_dir, exist_ok=True) # make an output directory to store the text file

    # Use PyPDFLoader to load text content
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)  #  text is splitted into chunks using RecursiveCharacterTextSplitter with a chunk size of 200 and an overlap of 50 characters.

    final_texts = ""

    # Process each page of the PDF
    doc = fitz.open(file)  # Open the PDF file with PyMuPDF
    for idx, page in enumerate(doc):
        if idx < len(pages):
            page_content = pages[idx].page_content
        else:
            page_content = ""
        if not page_content.strip():
            output_image_path = os.path.join(output_dir, f"page_{idx + 1}.png")  # Create an output image path for each page
            pix = page.get_pixmap()
            pix.save(output_image_path)

            ocr_text = ocr_image(output_image_path) # to extract text (if any) from the image
            page_content = ocr_text

        final_texts += page_content + "\n"


    document = Document(page_content=final_texts)
    chunks = text_splitter.split_documents([document])

    return final_texts

[2024/12/22 19:13:40] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [ ]:
pdf_path = r"/content/sample_data/UNIT III.pdf"
output_dir = r"/content/sample_data/OutputPDF"
pdf_text = file_processing(pdf_path, output_dir)  # an example use case

[2024/12/22 19:18:59] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.17888379096984863
[2024/12/22 19:18:59] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/12/22 19:18:59] ppocr DEBUG: rec_res num  : 0, elapsed : 1.6689300537109375e-06


In [ ]:
len(pdf_text.split()) # indicating pdf file has 5756 words

5756

In [ ]:
print(pdf_text)

Unit  3         Inheritance 
 
 
Inheritance 
 
Inheritance is one of the most useful and essential characteristics of object -oriented 
programming. The existing classes are the main components of inheritance. The new classes are 
created from existing ones. The properties of the existing classes are simply extended to the new 
classes. The new classes created by using such a method are known as derived classes, and the 
existing classes are known as base classes, as shown in the figure. The programmer can  define 
new member variables and functions in the derived class. The base class remains unchanged. The 
object of the derived class can access members of the base as well as derived classes. On the 
other hand, the object of the base class cannot access memb ers of the derived classes. The base 
class does not know about their subclasses. 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
The base class is also called super class, parent, or ancestor , and the derived class is 
called subclass, 

In [ ]:
4096 + 2048

6144

**LANGUAGE MODEL**
we are goinng to use unsloth's fastlanguage library to load models efficiently.
MODEL NAME is

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# installing necessary libraries

In [ ]:
max_seq_length = 2048
if (len(pdf_text.split()) > 2000):
  max_seq_length = 4096
if (len(pdf_text.split()) > 4000):
  max_seq_length = 6144

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = max_seq_length
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage and improve efficiency of loading of models

# creating tokenizer and model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # this model has been fine tuned for instruction tasks
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

In [ ]:
def generate_cheat_sheet(text):
  messages = get_messages(text)
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")  # using chat template to give clear system and user instructions
  max_token_to_be = 1024
  if (len(inputs[0]) > 4000): # if the limit of input tokens exceed 4000 we increase the size of output tokens generated
    max_token_to_be = 2048

  outputs = model.generate(inputs, max_new_tokens = max_token_to_be, use_cache = True)
  response = tokenizer.decode(outputs[0][len(inputs[0]) + 3:], skip_special_tokens = True)
  return response

def get_messages(text):
  system_prompt = """
  Create a comprehensive cheat sheet of about 800 words from the following technical text.
  Focus on:
  - Fundamental concepts
  - Key mathematical concepts if any otherwise don't mention it
  - Code snippets if any
  - Critical definitions
  - Important formulas and equations
  - Algorithms and their key steps
  - Fundamental principles

  Formatting Guidelines:
  - Use clear, concise sections
  - Include mathematical notation
  - Provide brief explanations
  - Highlight practical applications
  - Generate in Markdown
  """

  user_prompt = f"""
  Please summarize the following text \n {text}
  """

  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
  ]
  return messages



In [ ]:
generated_cheatsheet = generate_cheat_sheet(pdf_text) # cheat sheet generated

In [ ]:
print(generated_cheatsheet)



**Unit 3: Inheritance and Pointers**

**Inheritance**

Inheritance is a fundamental concept in object-oriented programming (OOP) that allows one class to inherit the properties and behavior of another class. The existing classes are the main components of inheritance, and the new classes are created from existing ones.

**Key Concepts:**

*   **Base Class:** The class from which the new class is derived.
*   **Derived Class:** The class that inherits the properties and behavior of the base class.
*   **Inheritance:** The process of creating a new class from an existing class.

**Types of Inheritance:**

*   **Single Inheritance:** A class can inherit from only one base class.
*   **Multiple Inheritance:** A class can inherit from more than one base class.
*   **Hierarchical Inheritance:** A single base class can be inherited by multiple derived classes.
*   **Multilevel Inheritance:** A derived class can inherit from another derived class.
*   **Hybrid Inheritance:** A combination of

In [ ]:
from IPython.display import Markdown, display

In [ ]:
len(generated_cheatsheet.split()) # number of words in the cheat sheet generated

784

In [ ]:
display(Markdown(generated_cheatsheet))



**Unit 3: Inheritance and Pointers**

**Inheritance**

Inheritance is a fundamental concept in object-oriented programming (OOP) that allows one class to inherit the properties and behavior of another class. The existing classes are the main components of inheritance, and the new classes are created from existing ones.

**Key Concepts:**

*   **Base Class:** The class from which the new class is derived.
*   **Derived Class:** The class that inherits the properties and behavior of the base class.
*   **Inheritance:** The process of creating a new class from an existing class.

**Types of Inheritance:**

*   **Single Inheritance:** A class can inherit from only one base class.
*   **Multiple Inheritance:** A class can inherit from more than one base class.
*   **Hierarchical Inheritance:** A single base class can be inherited by multiple derived classes.
*   **Multilevel Inheritance:** A derived class can inherit from another derived class.
*   **Hybrid Inheritance:** A combination of multiple and multilevel inheritance.

**Access Specifiers:**

*   **Public:** Members of the base class are accessible to the derived class.
*   **Private:** Members of the base class are not accessible to the derived class.
*   **Protected:** Members of the base class are accessible to the derived class, but not to other classes.

**Syntax:**

```cpp
class name_of_the_derived_class: access_specifiers name_of_the_base_class
{
    // member variables of new class (derived class)
}
```

**Example:**

```cpp
class B: public A
{
    // Members of class B
};
```

**Inheritance Definition:**

Inheritance is the procedure of creating a new class from one or more existing classes.

**Reusability:**

Reusability is achieved using inheritance, and it means the reuse of properties of the base class in the derived classes.

**Virtual Base Classes:**

To overcome the ambiguity occurring due to multipath inheritance, the C++ provides the keyword virtual.

**Pointers**

A pointer is a memory variable that stores a memory address.

**Key Concepts:**

*   **Pointer Declaration:** `int *x;`
*   **Indirection Operator:** `*x`
*   **Dereference Operator:** `*p`

**Features of Pointers:**

*   **Memory Space:** Pointers save memory space.
*   **Execution Time:** Execution time with pointers is faster.
*   **Memory Access:** Memory is accessed efficiently with the pointers.

**Arithmetic Operations with Pointers:**

*   **Increment:** `p = p + 1;`
*   **Decrement:** `p = p - 1;`
*   **Prefix:** `++p;`
*   **Postfix:** `p++;`

**Pointer to Pointer:**

A pointer to pointer is a pointer that stores the address of another pointer.

**Example:**

```cpp
int x = 10;
int *p = &x;
int **q = &p;
```

**Wild Pointers:**

Uninitialized pointers are known as wild pointers.

**This Pointer:**

When a member function is called, it is automatically passed an implicit argument that is a pointer to the invoking object.

**Example:**

```cpp
class integer
{
    int x;
public:
    void show_addr()
    {
        cout << "My Object's Address=" << this << "\n";
    }
};
```

**Polymorphism**

Polymorphism is a technique in which various forms of a single function can be defined and shared by various objects to perform an operation.

**Key Concepts:**

*   **Static Binding:** Matching of function call with the correct function definition happens at compile time.
*   **Dynamic Binding:** Matching of function call with the correct function definition happens at run time.

**Virtual Functions:**

A virtual function is a member function that is declared with in a base class and is redefined by derived class.

**Example:**

```cpp
class Base
{
public:
    virtual void display()
    {
        cout << "Base" << endl;
    }
};

class Derived: public Base
{
public:
    void display()
    {
        cout << "Derived" << endl;
    }
};
```

**Pure Virtual Functions:**

Pure virtual Functions are virtual functions with no definition.

**Example:**

```cpp
class Base
{
public:
    virtual void display() = 0;
};

class Derived: public Base
{
public:
    void display()
    {
        cout << "Derived" << endl;
    }
};
```

**Abstract Classes:**

Abstract Class is a class which contains at least one Pure Virtual function in it.

**Example:**

```cpp
class Base // Abstract base class
{
public:
    virtual void show() = 0; // Pure Virtual Function
};

class Derived: public Base
{
public:
    void show()
    {
        cout << "Implementation of Virtual Function in Derived class";
    }
};
```

**Run-Time Polymorphism Example:**

```cpp
class CPolygon
{
protected:
    int width, height;
public:
    void get(int first, int second)
    {
        width = first;
        height = second;
    }
    virtual int area() = 0;
};

class CRectangle: public CPolygon
{
public:
    int area()
    {
        return (width * height);
    }
};

class CTriangle: public CPolygon
{
public:
    int area()
    {
        return (width * height / 2);
    }
};

int main()
{
    CRectangle rectangle;
    CTriangle triangle;
    CPolygon *ptr_polygon;

    ptr_polygon = &rectangle;
    ptr_polygon->get(2, 2);
    cout << ptr_polygon->area() << endl;

    ptr_polygon = &triangle;
    ptr_polygon->get(2, 2);
    cout << ptr_polygon->area() << endl;

    return 0;
}
```

**Object Slicing:**

Virtual functions can be invoked using a pointer or reference. If we do so, object slicing takes place.

**Quiz Generation** Keyword extraction and sent tokenize to generate MCQs

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def extract_keywords_keybert(text, top_n = 20):
  """
  Extracts keywords from a text using KeyBERT

  Args:
  - text: str, the input text
  - top_n: int, the number of keywords to extract

  Returns:
  - List of keywords
  """
  kw_model = KeyBERT()
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range = (1, 2), stop_words = 'english', top_n = top_n)
  return [keyword[0] for keyword in keywords]


def get_passages(text):
  # split text into sentences
  sentences = sent_tokenize(text)

  # combine sentences into passages
  passages = []
  current_passage = ""
  total_words = sum([len(sentence.split()) for sentence in sentences])
  for sentence in sentences:
    if len(current_passage.split()) + len(sentence.split()) < int(total_words/10):  # if word limit of passage goes beyond 200 then it appends the current passage and starts a new one
        current_passage += " " + sentence
    else:
        passages.append(current_passage.strip())
        current_passage = sentence
  if current_passage:
      passages.append(current_passage.strip())

  return passages


def get_cleaned_text(text):
  cleaned_text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
  return cleaned_text


MCQ generation

In [ ]:
def get_message(keywords, passage):
  system_prompt = """
    You are a helpful assistant. You have deeo knowledge of computer science topics and are able to generate multiple choice questions based on a list of keywords and a given passage.
    Use your own domain knowledge of the subject on the list of keywords to generate only one single multiple choice question. The question should:
    - Be informative, testing foundational and deep knowledge
    - Include clear and concise questions with four options (a, b, c, d).
    - Ensure the questions are challenging and diverse in their difficulty.
  """

  system_prompt += """
      Additionally, the questions generated should adhere to the following format which is a python dictionary
      {'ques': 'Question generated', 'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d', 'ans': 'correct option'}
  """
  system_prompt += """
    For example, you may refer to the following:
    {'ques':'Why does Dijkstra's algorithm fail to work correctly with negative edge weights?', 'a':'Dijkstra's algorithm is designed to find the shortest path in unweighted graphs only.',
    'b':'Negative edge weights can cause the algorithm to update distances incorrectly, leading to incorrect shortest paths.', 'c':'The algorithm's priority queue cannot handle negative values.'
    'd':'Negative edge weights result in an infinite loop due to cyclic paths', 'ans':'b'}
  """

  user_prompt = f"""
  Please generate the a multiple choice question (MCQ) with the help of the keywords {keywords} and you may refer to the following passage for context \n {passage}

  """
  messages = [
      {"role":"system", "content":system_prompt},
      {"role":"user", "content":user_prompt}
  ]
  return messages


def generate_mcq(keywords, passage):
  messages = get_message(keywords, passage)
  inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to("cuda")
  outputs = model.generate(inputs, max_new_tokens = 1024, use_cache = True)
  response = tokenizer.decode(outputs[0][len(inputs[0]) + 3:], skip_special_tokens = True)
  return response

def get_all_ques(input_text):
  passages = get_passages(input_text)
  all_ques = []
  for passage in passages:
    keywords = extract_keywords_keybert(passage)
    ques = generate_mcq(keywords, passage)
    all_ques.append(ques)
  return all_ques

In [ ]:
questions = get_all_ques(generated_cheatsheet)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def Eval(text):  # to extract dictionaries of each question
  txt = ""
  flag = False
  for i in range(len(text)):
    char = text[i]
    if char == "{":
      flag = True
      txt += char
    elif char == "}":
      flag = False
      txt += char
    elif flag:
      txt += char
  return eval(txt) # to convert string to dictionary


def get_list_of_ques(questions):
  ques = []
  for ele in questions:
    ques.append(Eval(ele))
  return ques

In [ ]:
ques = get_list_of_ques(questions) # get questions in proper format of dictionaries

In [ ]:
for ele in ques:
  print(ele)

{'ques': 'What is the primary purpose of inheritance in object-oriented programming (OOP)?', 'a': 'To create new classes from scratch without any dependencies.', 'b': 'To allow one class to inherit the properties and behavior of another class.', 'c': 'To restrict the creation of new classes from existing ones.', 'd': 'To prevent the inheritance of properties from base classes.', 'ans': 'b'}
{'ques': 'What type of inheritance occurs when a derived class inherits from another derived class?', 'a': 'Single Inheritance', 'b': 'Multiple Inheritance', 'c': 'Multilevel Inheritance', 'd': 'Hierarchical Inheritance', 'ans': 'c'}
{'ques': 'Which access specifier in a base class makes its members accessible to the derived class, but not to other classes?', 'a': 'Public', 'b': 'Private', 'c': 'Protected', 'd': 'None of the above', 'ans': 'c'}
{'ques': 'What is the purpose of using the syntax `class name_of_the_derived_class: access_specifiers name_of_the_base_class` in C++?', 'a': 'To declare a ne

Exporting cheat sheet to PDF

In [ ]:
!pip install fpdf markdown beautifulsoup4

from fpdf import FPDF
from IPython.display import Markdown, display
import markdown
from bs4 import BeautifulSoup

def markdown_to_plain_text(md_text):
    """
    Convert Markdown text to plain text using BeautifulSoup.
    """
    # Convert Markdown to HTML
    html_content = markdown.markdown(md_text)

    # Use BeautifulSoup to extract plain text
    soup = BeautifulSoup(html_content, "html.parser")
    plain_text = soup.get_text()

    return plain_text

def string_to_pdf(text, output_filename):
    """
    Save plain text to a PDF while preserving formatting.
    """
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Adding each line of text while respecting indentation and formatting
    for line in text.split("\n"):
        pdf.multi_cell(0, 10, line)

    pdf.output(output_filename)



  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=950fe9f554a6b1485fcc5ab552c95d0ff203cd7c7acdf3d59fa2e31bf933ac49
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
plain_text = markdown_to_plain_text(generated_cheatsheet)
output_file = "cheat_sheet_output.pdf"
string_to_pdf(plain_text, output_file)
# storing pdf in output file

In [ ]:
print(plain_text)

Unit 3: Inheritance and Pointers
Inheritance
Inheritance is a fundamental concept in object-oriented programming (OOP) that allows one class to inherit the properties and behavior of another class. The existing classes are the main components of inheritance, and the new classes are created from existing ones.
Key Concepts:

Base Class: The class from which the new class is derived.
Derived Class: The class that inherits the properties and behavior of the base class.
Inheritance: The process of creating a new class from an existing class.

Types of Inheritance:

Single Inheritance: A class can inherit from only one base class.
Multiple Inheritance: A class can inherit from more than one base class.
Hierarchical Inheritance: A single base class can be inherited by multiple derived classes.
Multilevel Inheritance: A derived class can inherit from another derived class.
Hybrid Inheritance: A combination of multiple and multilevel inheritance.

Access Specifiers:

Public: Members of the bas

In [1]:
!git add MCQ+CheatSheet+outputPDF.ipynb

fatal: not a git repository (or any of the parent directories): .git


In [2]:
!git commit -m "Updated notebook with documentation"

fatal: not a git repository (or any of the parent directories): .git
